<a href="https://colab.research.google.com/github/lakhbir-05/Thapar-Summer-School-2025-Hack-III/blob/main/competition3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --force-reinstall --no-cache-dir numpy==1.26.4 scipy==1.11.4 pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 191.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 270.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 215.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 66.5 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2

In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 17.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import optuna

# === Load Datasets ===
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# === Drop unnecessary columns ===
train = train.drop(columns=['Row#'])
test = test.drop(columns=['Row#'])

# === Prepare Features and Target ===
X = train.drop(columns=['output'])
y = train['output']
X_test = test.drop(columns=['id'])

# === Scale Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# === Train/Validation Split ===
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

# === Optuna + RandomForest Using R² Score ===
def run_optuna_random_forest():
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 5, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
        }
        model = RandomForestRegressor(random_state=42, **params)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        return -r2_score(y_val, preds)  # Maximize R² by minimizing its negative

    # Run Optuna Study
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=200)  # Increase to 200–300 for better tuning

    print("✅ Best Parameters Found by Optuna:", study.best_params)
    print("✅ Best Validation R² Score:", -study.best_value)

    # Train on full data with best params
    best_model = RandomForestRegressor(random_state=42, **study.best_params)
    best_model.fit(X_scaled, y)
    return best_model

# === Run the model optimization and training ===
model = run_optuna_random_forest()

# === Predict on Test Data ===
final_preds = model.predict(X_test_scaled)

# === Save Final Submission File ===
#submission = pd.DataFrame({
   # 'id': test['id'],
    #'output': final_preds
#})
#submission.to_csv('lakhbir_1186', index=False)
#print("✅ file created")
# Create submission file
sample_submission['output'] = final_preds
sample_submission.to_csv('lakhbir_1187.csv', index=False)
print("lakhbir_1187.csv created!")


[I 2025-07-07 12:11:32,010] A new study created in memory with name: no-name-4ac89530-3734-49fb-8f64-97fb5df84432
[I 2025-07-07 12:11:53,163] Trial 0 finished with value: -0.9153417184721346 and parameters: {'n_estimators': 403, 'max_depth': 27, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: -0.9153417184721346.
[I 2025-07-07 12:12:10,412] Trial 1 finished with value: -0.9186533101669049 and parameters: {'n_estimators': 393, 'max_depth': 28, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 1 with value: -0.9186533101669049.
[I 2025-07-07 12:12:15,702] Trial 2 finished with value: -0.9197157628444274 and parameters: {'n_estimators': 137, 'max_depth': 16, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 2 with value: -0.9197157628444274.
[I 2025-07-07 12:12:20,872] Trial 3 finished with value: -0.9177923029398706 and parameters: {'n_estimators': 118, 'max_depth': 21, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 2 wi

✅ Best Parameters Found by Optuna: {'n_estimators': 147, 'max_depth': 5, 'min_samples_split': 7, 'min_samples_leaf': 5}
✅ Best Validation R² Score: 0.923883285198505
lakhbir_1187.csv created!


In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual

In [ ]:
!pip install catboost xgboost autogluon optuna scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score
import optuna
import xgboost as xgb

# === Load Datasets ===
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# === Drop unnecessary columns ===
train.drop(columns=['Row#'], inplace=True)
test.drop(columns=['Row#'], inplace=True)

# === Prepare Features and Target ===
X = train.drop(columns=['output'])
y = train['output']
X_test = test.drop(columns=['id'])

# === Feature Engineering ===
def feature_engineering(X_df, X_test_df):
    # Polynomial Features (only on numeric)
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X_df)
    X_test_poly = poly.transform(X_test_df)

    return X_poly, X_test_poly

X_poly, X_test_poly = feature_engineering(X, X_test)

# === Scale Features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)
X_test_scaled = scaler.transform(X_test_poly)

# === Optuna + XGBoost with K-Fold Cross-Validation (R² Score) ===
def run_optuna_xgboost_kfold():
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)
        }

        kf = KFold(n_splits=6, shuffle=True, random_state=42)
        r2_scores = []

        for train_idx, val_idx in kf.split(X_scaled):
            X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
            y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

            model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **params)
            model.fit(X_train_fold, y_train_fold)
            preds = model.predict(X_val_fold)
            r2 = r2_score(y_val_fold, preds)
            r2_scores.append(r2)

        return -np.mean(r2_scores)  # Minimize the negative mean R²

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    print("✅ Best Parameters:", study.best_params)
    print("✅ Best Mean Validation R² Score (CV):", -study.best_value)

    best_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **study.best_params)
    best_model.fit(X_scaled, y)

    train_preds = best_model.predict(X_scaled)
    final_r2 = r2_score(y, train_preds)
    print("🎯 Final R² Score on Full Training Data:", final_r2)

    return best_model

# === Run Optimization and Train Final Model ===
model = run_optuna_xgboost_kfold()

# === Predict on Test Data ===
final_preds = model.predict(X_test_scaled)

# === Save Final Submission File ===
sample_submission['output'] = final_preds
sample_submission.to_csv('lakhbir_xgboost_kfold.csv', index=False)
print("✅ lakhbir_xgboost_kfold.csv created!")


[I 2025-07-08 15:26:42,055] A new study created in memory with name: no-name-b9f60886-6bec-4c8f-9039-717d59704d83
[I 2025-07-08 15:31:16,354] Trial 0 finished with value: -0.9055507503445087 and parameters: {'n_estimators': 228, 'max_depth': 15, 'learning_rate': 0.14879178799465856, 'subsample': 0.9178854339845965, 'colsample_bytree': 0.7162738843325035, 'gamma': 1.3397176185803006, 'reg_alpha': 3.307685745505677, 'reg_lambda': 0.2529083456591785}. Best is trial 0 with value: -0.9055507503445087.
[I 2025-07-08 15:31:33,779] Trial 1 finished with value: -0.9225754768262413 and parameters: {'n_estimators': 154, 'max_depth': 5, 'learning_rate': 0.042143471214329487, 'subsample': 0.7003371818563833, 'colsample_bytree': 0.9792290097320004, 'gamma': 3.9842564411461217, 'reg_alpha': 1.9554332493185052, 'reg_lambda': 2.0515358301576954}. Best is trial 1 with value: -0.9225754768262413.
[I 2025-07-08 15:40:33,153] Trial 2 finished with value: -0.9102947636672546 and parameters: {'n_estimators':

✅ Best Parameters: {'n_estimators': 295, 'max_depth': 4, 'learning_rate': 0.02431382676462817, 'subsample': 0.7958715573027214, 'colsample_bytree': 0.5692859500509893, 'gamma': 1.6809686871595524, 'reg_alpha': 0.6969687356722283, 'reg_lambda': 2.334498829670464}
✅ Best Mean Validation R² Score (CV): 0.9232149070956792
🎯 Final R² Score on Full Training Data: 0.9362199883152894
✅ lakhbir_xgboost_kfold.csv created!


In [ ]:
!pip install xgboost

In [ ]:
# prompt: use autogluon

!pip install autogluon
from autogluon.tabular import TabularPredictor

# Assuming train and test DataFrames are already loaded and processed
# You might need to adjust column names if they are different

# Define the target variable
label = 'output'

# Initialize TabularPredictor
# 'best_quality' setting trains a variety of models and ensembles them
predictor = TabularPredictor(label=label, eval_metric='r2').fit(train, presets='best_quality')

# Make predictions on the test data
predictions = predictor.predict(test)

# Convert predictions to integer if required
predictions = predictions.round().astype(int)

# Save the submission file
# Assuming sample_submission is already loaded and has an 'id' column
submission_autogluon = sample_submission.copy()
submission_autogluon['output'] = predictions
submission_autogluon.to_csv('autogluon_submission.csv', index=False)

print("✅ autogluon_submission.csv created!")

# You can also get leaderboad to see the performance of different models trained by AutoGluon
leaderboard = predictor.leaderboard(silent=True)
leaderboard


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

No path specified. Models will be saved in: "AutogluonModels/ag-20250709_011357"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.88 GB / 12.67 GB (85.8%)
Disk Space Avail:   65.04 GB / 107.72 GB (60.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be

(_ray_fit pid=7151) [1000]	valid_set's l2: 141486	valid_set's r2: 0.922797
(_ray_fit pid=7307) [1000]	valid_set's l2: 139286	valid_set's r2: 0.919015 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(_dystack pid=6792) 	0.921	 = Validation score   (r2)
(_dystack pid=6792) 	57.85s	 = Training   runtime
(_dystack pid=6792) 	1.39s	 = Validation runtime
(_dystack pid=6792) Fitting model: LightGBM_BAG_L1 ... Training model for up to 528.08s of the 826.24s of remaining time.
(_dystack pid=6792) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.12%)
(_dystack pid=6792) 	0.9234	 = Validation score   (r2)
(_dystack pid=6792) 	44.39s	 = Training   runtime
(_dystack pid=6792) 	0.22s	 = Validation runtime
(_dystack pid=6792) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 478.94s of the 777.11s of remaining time.
(_dystack pid=6792) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-lear

✅ autogluon_submission.csv created!


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.925805,r2,8.843771,1841.899952,0.002394,0.504927,3,True,27
1,WeightedEnsemble_L2,0.925383,r2,2.315296,1375.600379,0.001468,0.649298,2,True,16
2,CatBoost_r177_BAG_L2,0.924607,r2,7.281036,1711.187379,0.110571,59.205927,2,True,26
3,CatBoost_BAG_L2,0.924477,r2,7.244207,1720.667668,0.073743,68.686216,2,True,20
4,LightGBM_BAG_L2,0.924329,r2,7.343542,1703.377781,0.173077,51.396329,2,True,18
5,LightGBM_BAG_L1,0.924055,r2,0.147568,43.313611,0.147568,43.313611,1,True,4
6,LightGBMXT_BAG_L2,0.923771,r2,7.574153,1701.804771,0.403688,49.823320,2,True,17
7,CatBoost_BAG_L1,0.923742,r2,0.078168,47.162726,0.078168,47.162726,1,True,6
8,CatBoost_r177_BAG_L1,0.923625,r2,0.075666,42.804476,0.075666,42.804476,1,True,12
9,NeuralNetFastAI_BAG_L2,0.923398,r2,7.580609,1788.825384,0.410145,136.843932,2,True,22
